In [2]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings(action='ignore')
# warnings.filterwarnings(action='default')

In [3]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../data/'

In [15]:
train_df = pd.read_csv(data_path+'train.csv')
test_df = pd.read_csv(data_path+'test.csv')
submission_df = pd.read_csv(data_path+'sample_submission.csv')
target_df = pd.read_csv(data_path+'revealed_targets.csv')

In [16]:
# null 값 처리
train_df['far_price'] = train_df['far_price'].fillna(0)
train_df['near_price'] = train_df['near_price'].fillna(1)
train_df.isnull().sum()

In [23]:
cols_group_by = ['date_id', 'seconds_in_bucket']
cols_fill_nan = [
    'imbalance_size', 'reference_price', 'matched_size', 'wap',
    'bid_price', 'bid_size', 'ask_price', 'ask_size',
    'stock_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag']
train_grouped_median = train_df.groupby(cols_group_by)[cols_fill_nan].transform('median')
train_df[cols_fill_nan] = train_df[cols_fill_nan].fillna(train_grouped_median)
train_df.loc[train_df.isnull().any(axis=1)] # target 88

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,1381981.10,0,0.999950,9723622.86,0.0,1.0,0.999688,0.0,1.000242,0.0,1.000000,NaN,1925,35_0_131
369700,131,35,10,1371886.54,0,1.000252,9961197.49,0.0,1.0,0.999969,0.0,1.000485,0.0,1.000223,NaN,1926,35_10_131
369892,131,35,20,1331838.54,0,1.000122,9999133.11,0.0,1.0,0.999883,0.0,1.000328,0.0,1.000149,NaN,1927,35_20_131
370084,131,35,30,1350584.58,0,0.999910,10133596.07,0.0,1.0,0.999757,0.0,1.000186,0.0,0.999971,NaN,1928,35_30_131
370276,131,35,40,1327284.70,0,0.999926,10133596.07,0.0,1.0,0.999758,0.0,1.000203,0.0,0.999984,NaN,1929,35_40_131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225338,158,388,510,339056.84,0,1.001074,19640140.62,0.0,1.0,1.000867,0.0,1.001250,0.0,1.001027,NaN,21391,388_510_158
4225538,158,388,520,437746.70,0,1.001183,19767348.90,0.0,1.0,1.001007,0.0,1.001290,0.0,1.001163,NaN,21392,388_520_158
4225738,158,388,530,483535.46,0,1.000994,19800447.29,0.0,1.0,1.000905,0.0,1.001187,0.0,1.001032,NaN,21393,388_530_158
4225938,158,388,540,453894.62,0,1.000989,20020720.89,0.0,1.0,1.000824,0.0,1.001024,0.0,1.000911,NaN,21394,388_540_158


In [36]:
# (stock_id=131, date_id=35), (stock_id=158, date_id=388) target이 없으므로 제거
train_df = train_df.drop(index=train_df.loc[((train_df['stock_id']==131) & (train_df['date_id']==35))  | ((train_df['stock_id']==158) & (train_df['date_id']==388))].index)
train_df[train_df.isnull().any(axis=1)]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
3555081,101,328,0,3683316.08,0,0.999681,7405651.01,0.0,1.0,0.999647,0.0,1.000356,0.0,1.0,NaN,18040,328_0_101
4764999,19,438,0,2201071.62,-1,1.000146,6841177.82,0.0,1.0,0.999749,0.0,1.000277,0.0,1.0,NaN,24090,438_0_19


In [49]:
# stock_id=101, date_id=328, seconds_in_bucket=0의 target값을 stock_id=101, date_id=328, seconds_in_bucket=1의 target값으로 세팅
train_df.loc[train_df['row_id']=='328_0_101', 'target'] = train_df.loc[(train_df['stock_id']==101) & (train_df['time_id']==18041), 'target'].values
# stock_id=19, date_id=438, seconds_in_bucket=0의 target값을 stock_id=19, date_id=438, seconds_in_bucket=1의 target값으로 세팅
train_df.loc[train_df['row_id']=='438_0_19', 'target'] = train_df.loc[(train_df['stock_id']==19) & (train_df['time_id']==24091), 'target'].values
train_df.isnull().sum()

stock_id                   0
date_id                    0
seconds_in_bucket          0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
far_price                  0
near_price                 0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
row_id                     0
dtype: int64

In [67]:
from tqdm.auto import tqdm

X_all = None
y_all = None
for stock_id in tqdm(train_df['stock_id'].unique()):
    stock_data = train_df[train_df['stock_id']==stock_id].drop(columns='target').values
    y_data = train_df.loc[train_df['stock_id']==stock_id, 'target'].to_numpy()
    stock_data = stock_data.reshape(-1, 55, 16).transpose(0,2,1)
    y_data = y_data.reshape(-1,55,1).transpose(0,2,1).squeeze() # (95234, 1, 55) =squeeze=> (95234, 55)
    if X_all is None:
        X_all = stock_data
        y_all = y_data
    else:
        X_all = np.concatenate((X_all, stock_data), axis=0)
        y_all = np.concatenate((y_all, y_data), axis=0)
X_all.shape, y_all.shape

  0%|          | 0/200 [00:00<?, ?it/s]

((95234, 16, 55), (95234, 55))

(pandas.core.frame.DataFrame, (5237980, 16), numpy.ndarray, (5237980,))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  70,  71,  72,  73,  74,  75,  76,  77,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100, 101, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 151, 152, 154, 155, 157, 158, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176,
       177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 18